# Izrada AI agenata s trajnom memorijom koristeći Cognee

Ovaj notebook pokazuje kako izraditi inteligentne AI agente s naprednim mogućnostima memorije koristeći [**cognee**](https://www.cognee.ai/) - otvoreni izvor AI memorije koji kombinira grafove znanja, semantičko pretraživanje i upravljanje sesijama za stvaranje AI sustava svjesnih konteksta.

## 🎯 Ciljevi učenja

Na kraju ovog vodiča, razumjet ćete kako:
- **Izraditi grafove znanja podržane ugrađenim podacima**: Pretvoriti nestrukturirani tekst u strukturirano, upitno znanje
- **Implementirati memoriju sesije**: Stvoriti višestruke razgovore s automatskim zadržavanjem konteksta
- **Pohraniti razgovore**: Opcionalno spremiti važne interakcije u dugoročnu memoriju za buduću upotrebu
- **Postavljati upite koristeći prirodni jezik**: Pristupiti i koristiti povijesni kontekst u novim razgovorima
- **Vizualizirati memoriju**: Istražiti odnose u grafu znanja vašeg agenta


## 🏗️ Što ćete izraditi

U ovom vodiču, stvorit ćemo **Asistenta za kodiranje** s trajnom memorijom koji:

### 1. **Izgradnja baze znanja**
   - Uključuje informacije o profilu i stručnosti programera
   - Obrađuje principe i najbolje prakse programiranja u Pythonu
   - Pohranjuje povijesne razgovore između programera i AI asistenata

### 2. **Razgovori svjesni sesije**
   - Održava kontekst kroz više pitanja u istoj sesiji
   - Automatski sprema svaki par pitanja/odgovora za učinkovito dohvaćanje
   - Pruža koherentne, kontekstualne odgovore na temelju povijesti razgovora

### 3. **Dugoročna memorija**
   - Pohranjuje važne razgovore u dugoročnu memoriju
   - Dohvaća relevantne uspomene iz baze znanja i prošlih sesija kako bi informirao nove interakcije
   - Gradi rastuću bazu znanja koja se s vremenom poboljšava

### 4. **Inteligentno dohvaćanje memorije**
   - Koristi semantičko pretraživanje svjesno grafa za pronalaženje relevantnih informacija u pohranjenoj bazi znanja
   - Filtrira pretraživanja prema podskupinama podataka (informacije o programeru vs. principi)
   - Kombinira više izvora podataka kako bi pružio sveobuhvatne odgovore


## 📋 Preduvjeti i postavljanje

### Sistemski zahtjevi

Prije početka, osigurajte da imate:

1. **Python okruženje**
   - Python 3.9 ili noviji
   - Virtualno okruženje (preporučeno)
   
2. **Redis Cache** (Potrebno za upravljanje sesijama)
   - Lokalni Redis: `docker run -d -p 6379:6379 redis`
   - Ili koristite upravljanu Redis uslugu
   
3. **Pristup LLM API-ju**
   - OpenAI API ključ ili drugi pružatelji usluga (pogledajte [dokumentaciju](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Konfiguracija baze podataka**
   - Nema potrebne konfiguracije prema zadanim postavkama. Cognee koristi baze podataka temeljene na datotekama (LanceDB i Kuzu)
   - Opcionalno, možete postaviti Azure AI Search kao spremište vektora (pogledajte [dokumentaciju](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Konfiguracija okruženja

Kreirajte `.env` datoteku u direktoriju vašeg projekta s sljedećim varijablama:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Razumijevanje Cogneejeve memorijske arhitekture

### Kako Cognee funkcionira

Cognee pruža sofisticirani memorijski sustav koji nadilazi jednostavno pohranjivanje ključ-vrijednost:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Ključne komponente:

1. **Graf znanja**: Pohranjuje entitete, odnose i semantičke veze
2. **Vektorski ugrađaji**: Omogućuju semantičko pretraživanje kroz sve pohranjene informacije
3. **Predmemorija sesije**: Održava kontekst razgovora unutar i između sesija
4. **NodeSets**: Organiziraju podatke u logičke kategorije za ciljani dohvat

### Vrste memorije u ovom vodiču:

- **Trajna memorija**: Dugoročno pohranjivanje u grafu znanja
- **Memorija sesije**: Privremeni kontekst razgovora u Redis predmemoriji
- **Semantička memorija**: Pretraživanje temeljeno na sličnosti vektora kroz sve podatke


## 📦 Instalirajte potrebne pakete

Instalirajte Cognee s podrškom za Redis za upravljanje sesijama:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicijalizirajte okruženje i učitajte biblioteke

Provjerite:
1. Redis je pokrenut (npr. putem Dockera: `docker run -d -p 6379:6379 redis`)
2. Varijable okruženja su postavljene prije uvoza modula za predmemoriju
3. Ako je potrebno, ponovno pokrenite kernel i izvršite ćelije redoslijedom

Sljedeća ćelija će:
1. Učitati varijable okruženja iz `.env`
2. Konfigurirati Cognee s vašim LLM postavkama
3. Omogućiti predmemoriju za upravljanje sesijama
4. Provjeriti jesu li svi dijelovi ispravno povezani


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfiguriranje direktorija za pohranu

Cognee koristi dva odvojena direktorija za svoje operacije:
- **Korijenski direktorij podataka**: Pohranjuje unesene dokumente i obrađene podatke
- **Korijenski direktorij sustava**: Sadrži bazu podataka grafova znanja i sistemske metapodatke

Za ovaj vodič ćemo kreirati odvojene direktorije na sljedeći način:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Resetiranje stanja memorije

Prije nego što počnemo graditi naš sustav memorije, osigurajmo da počinjemo od nule.

> 💡 **Savjet**: Možete preskočiti ovaj korak ako želite sačuvati postojeće memorije iz prethodnih pokretanja kada kasnije koristite ovu bilježnicu.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Dio 1: Izgradnja baze znanja

### Izvori podataka za našeg asistenta za programere

Koristit ćemo tri vrste podataka kako bismo stvorili sveobuhvatnu bazu znanja:

1. **Profil programera**: Osobna stručnost i tehnička pozadina
2. **Najbolje prakse za Python**: Zen of Python s praktičnim smjernicama
3. **Povijesni razgovori**: Prošli Q&A razgovori između programera i AI asistenata

Ova raznolika baza podataka omogućuje našem agentu da:
- Razumije tehnički kontekst korisnika
- Primjenjuje najbolje prakse u preporukama
- Uči iz prethodnih uspješnih interakcija


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Pretvorba podataka u graf znanja

Sada ćemo naš sirovi tekst pretvoriti u strukturiranu memoriju. Ovaj proces:

1. **Dodaje podatke u NodeSets**: Organiziranje informacija u logične kategorije
   - `developer_data`: Profil programera i razgovori
   - `principles_data`: Najbolje prakse i smjernice za Python

2. **Pokreće Cognify Pipeline**: Izvlači entitete, odnose i stvara ugrađivanja
   - Prepoznaje ključne pojmove
   - Stvara semantičke veze između povezanih informacija
   - Generira vektorska ugrađivanja

Ovo može potrajati nekoliko trenutaka dok LLM obrađuje tekst i gradi strukturu grafa:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Vizualizirajte graf znanja

Istražimo strukturu našeg grafa znanja. Vizualizacija prikazuje:
- **Čvorove**: Entiteti izdvojeni iz teksta (pojmovi, tehnologije, osobe)
- **Rubove**: Odnosi i poveznice između entiteta
- **Klastere**: Povezani pojmovi grupirani prema semantičkoj sličnosti

Otvorite generiranu HTML datoteku u svom pregledniku kako biste interaktivno istražili graf:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Obogatite memoriju s Memify

Funkcija `memify()` analizira graf znanja i generira inteligentna pravila o podacima. Ovaj proces:
- Prepoznaje obrasce i najbolje prakse
- Stvara praktične smjernice na temelju sadržaja
- Uspostavlja odnose između različitih područja znanja

Ova pravila pomažu agentu da donosi bolje informirane odluke prilikom odgovaranja na pitanja. Snimanje druge vizualizacije omogućuje vam usporedbu kako se graf zgušnjava nakon obogaćivanja.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Dio 2: Inteligentno dohvaćanje memorije

### Demonstracija 1: Integracija znanja iz više dokumenata

Sada kada je naš graf znanja izgrađen, testirajmo kako Cognee kombinira informacije iz više izvora kako bi odgovorio na složena pitanja.

Prvi upit pokazuje:
- **Semantičko razumijevanje**: Pronalaženje relevantnih pojmova čak i kada nisu izričito spomenuti
- **Križno referenciranje**: Kombiniranje profila programera s principima Pythona
- **Kontekstualno zaključivanje**: Primjena najboljih praksi na specifične implementacije

### Demonstracija 2: Filtrirano pretraživanje pomoću NodeSets

Drugi upit pokazuje kako ciljati određene podskupove grafa znanja:
- Koristi parametar `node_name` za pretraživanje samo unutar `principles_data`
- Pruža fokusirane odgovore iz specifičnog područja znanja
- Korisno kada trebate informacije specifične za određenu domenu


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Dio 3: Postavljanje upravljanja sesijama

### Omogućavanje memorije razgovora

Upravljanje sesijama ključno je za održavanje konteksta tijekom više interakcija. Ovdje ćemo:

1. **Inicijalizirati korisnički kontekst**: Stvoriti ili dohvatiti korisnički profil za praćenje sesije
2. **Konfigurirati mehanizam predmemorije**: Povezati se s Redisom za pohranu povijesti razgovora
3. **Omogućiti varijable sesije**: Postaviti kontekstualne varijable koje ostaju postojane tijekom upita

> ⚠️ **Važno**: Ovo zahtijeva da Redis radi i da je `CACHING=true` postavljeno u vašem okruženju


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Pomoćna funkcija: Pregled povijesti sesije

Ova pomoćna funkcija omogućuje nam pregled povijesti razgovora pohranjene u Redis-u. Korisna je za:
- Otklanjanje pogrešaka u upravljanju sesijama
- Provjeru da li se razgovori spremaju u predmemoriju
- Razumijevanje konteksta koji je dostupan agentu


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesija 1: Laboratorij za podršku asinkronom radu — Prvo pitanje

Započnite sesiju `async-support-lab` postavljanjem pitanja o obrascima asyncio prilagođenim telemetriji za masivni web scraper. Graf već poznaje asyncio, aiohttp i prakse praćenja, pa bi odgovor trebao odražavati prethodne razgovore dok prilagođava odgovor novom upitu.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Pregled memorije sesije 1 nakon prve razmjene

Pokretanjem `show_history(session_1)` odmah nakon početnog pitanja potvrđuje se da je Cognee zapisao i upit i odgovor u Redis. Trebali biste vidjeti jedan unos s uputama za istovremeni rad.


In [ ]:
await show_history(session_1)

## Sesija 1: Praćenje modela podataka

Zatim se pitamo: "Kada bih trebao odabrati dataclasses umjesto Pydantic?" koristeći isti ID sesije. Cognee bi trebao povezati Python principe s prethodnim razgovorima o FastAPI-ju kako bi pružio nijansirane savjete—pokazujući da se kontekst prenosi unutar imenovane sesije.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Potvrdite da Povijest Sesije 1 Sadrži Oba Kruga

Još jedan poziv `show_history(session_1)` trebao bi prikazati dva unosa pitanja i odgovora. Ovo odgovara koraku "reprodukcije memorije" u Mem0 laboratoriju i dokazuje da dodatni krugovi proširuju isti transkript.


In [ ]:
await show_history(session_1)

## Sesija 2: Pregled dizajna — Nova sesija

Kako bismo pokazali izolaciju između niti, pokrećemo `design-review-session` i tražimo smjernice za bilježenje pregleda incidenata. Iako je osnovna baza znanja ista, novi ID sesije drži transkripte odvojenima.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Pregled Sesije 2 Povijest

`show_history(session_2)` treba prikazati samo par upita/odgovora za dizajnersku recenziju. Usporedite to sa Sesijom 1 kako biste istaknuli kako Cognee održava neovisne transkripte dok ponovno koristi zajednički graf znanja.


In [ ]:
await show_history(session_2)

## Sažetak

Čestitamo! Upravo ste svom pomoćniku za kodiranje dodali pravi sloj dugoročne memorije koji pokreće Cognee.

U ovom vodiču uzeli ste sirovi sadržaj za razvoj (kod, dokumentaciju, razgovore) i pretvorili ga u graf + vektorsku memoriju koju vaš agent može pretraživati, analizirati i kontinuirano poboljšavati.

Što ste naučili

1. **Od sirovog teksta do AI memorije**: Kako Cognee unosi nestrukturirane podatke i pretvara ih u inteligentnu, pretraživu memoriju koristeći kombiniranu arhitekturu vektora + graf znanja.

2. **Obogaćivanje grafa pomoću memify**: Kako nadmašiti osnovno stvaranje grafa i koristiti memify za dodavanje izvedenih činjenica i bogatijih odnosa na vaš postojeći graf.

3. **Višestruke strategije pretraživanja**: Kako pretraživati memoriju koristeći različite vrste pretraživanja (Q&A svjesno grafa, RAG-stil dovršavanja, uvidi, sirovi dijelovi, pretraživanje koda itd.) ovisno o potrebama vašeg agenta.

4. **Vizualno istraživanje**: Kako pregledati i otkloniti pogreške u onome što je Cognee izgradio koristeći vizualizacije grafa i Cognee UI, tako da možete vidjeti kako je znanje strukturirano.

5. **Memorija svjesna sesije**: Kako kombinirati kontekst po sesiji s trajnom semantičkom memorijom kako bi agenti mogli pamtiti kroz različite pokretanja bez curenja informacija između korisnika.


## Ključne točke
1. Memorija kao graf znanja podržan ugradnjama

    - **Strukturirano razumijevanje**: Cognee kombinira spremište vektora i graf spremište kako bi vaši podaci bili pretraživi po značenju i povezani odnosima. Cognee koristi baze podataka temeljene na datotekama prema zadanim postavkama (LanceDB za vektore, Kuzu za graf baze podataka).

    - **Pretraživanje svjesno odnosa**: Odgovori se mogu temeljiti ne samo na "sličnom tekstu", već i na tome kako su entiteti povezani.

    - **Živa memorija**: Sloj memorije se razvija, raste i ostaje pretraživ kao jedan povezani graf.

2. Načini pretraživanja i zaključivanja
    - **Hibridno pretraživanje**: pretraživanje kombinira sličnost vektora, strukturu grafa i zaključivanje LLM-a, od pretraživanja sirovih dijelova do odgovaranja na pitanja svjesnih grafa.

    - **Prilagodite način zadatku**: Koristite načine u stilu dovršavanja kada želite odgovore u prirodnom jeziku, a načine za dijelove/sažetke/grafove kada vaš agent treba sirovi kontekst ili za vlastito zaključivanje.

3. Personalizirani agenti svjesni sesije
    - **Kontekst sesije + dugoročna memorija**: Cognee drži kratkoročni kontekst "niti" odvojen od dugoročne memorije na razini korisnika ili organizacije.

## Primjene u stvarnom svijetu

1. **Vertikalni AI agenti**

    Koristite uzorak iz ove bilježnice za pokretanje domenski pametnih kopilota koji se oslanjaju na Cognee kao jezgru za pretraživanje i zaključivanje:

- **Kopiloti za programere**: Pregled koda, analiza incidenata i asistenti za arhitekturu koji prolaze kroz kod, API-je, dizajnerske dokumente i tikete kao jedan graf memorije.

- **Kopiloti usmjereni na korisnike**: Agenti za podršku ili uspjeh koji koriste dokumentaciju proizvoda, često postavljana pitanja, CRM bilješke i prošle tikete s pretraživanjem svjesnim grafa i citiranim odgovorima.

- **Interni stručni kopiloti**: Asistenti za politiku, pravne ili sigurnosne poslove koji zaključuju na temelju međusobno povezanih pravila, smjernica i povijesnih odluka umjesto izoliranih PDF-ova.

    Cognee je izričito pozicioniran kao trajna, točna memorija za AI agente, pružajući živi graf znanja koji se integrira iza vašeg agenta i zamjenjuje ad-hoc kombinacije spremišta vektora i prilagođenog koda za grafove.

2. **Ujedinjavanje podataka iz silosa u jednu memoriju**

    Isti pristup također vam pomaže izgraditi sloj ujedinjene memorije preko raspršenih izvora:

- **Od silosa do jednog grafa**: Učitajte strukturirane (npr. baze podataka) i nestrukturirane podatke (npr. dokumente, razgovore) u jedan graf podržan ugradnjama, umjesto odvojenih indeksa za svaki sustav.

- **Zaključivanje preko izvora s citatima**: Provedite višekorakno zaključivanje nad svime—"spojite" dnevnike, metrike i dokumente putem grafa—i još uvijek vratite utemeljene odgovore s dokazima.

- **Čvorišta znanja**: Za domene poput bankarstva ili obrazovanja, Cognee se već koristi za ujedinjavanje PDF-ova, internih sustava i podataka aplikacija u jedan graf znanja s vektorima kako bi agenti mogli odgovarati na pitanja s preciznim, citiranim kontekstom.

## Sljedeći koraci

Implementirali ste osnovnu petlju memorije. Evo prirodnih proširenja koja možete isprobati sami (pogledajte [Cognee dokumentaciju](https://docs.cognee.ai/) za detalje):

1. **Eksperimentirajte s vremenskom sviješću**: Uključite vremensku funkciju kako biste izdvojili događaje i vremenske oznake iz teksta.

2. **Uvedite zaključivanje vođeno ontologijom**: Definirajte OWL ontologiju za svoju domenu. Koristite Cognee-ovu podršku za ontologiju kako bi izdvojeni entiteti i odnosi bili utemeljeni u toj shemi, poboljšavajući kvalitetu grafa i odgovore specifične za domenu.

3. **Dodajte povratnu petlju**: Dopustite Cognee-u da prilagodi težine rubova grafa na temelju stvarnih povratnih informacija korisnika, tako da se pretraživanje s vremenom poboljšava umjesto da ostane statično.

4. **Prilagodite za personalizaciju i ponašanje sesije**: Koristite korisničke ID-ove, stanare i skupove podataka kako biste svakom korisniku ili timu dali vlastiti pogled na zajednički motor memorije.

5. **Proširite na složenije agente**: Povežite Cognee s okvirima za agente kako biste izgradili sustave s više agenata koji svi dijele isti sloj memorije. *Microsoft Agent Framework x Cognee dodatak dolazi uskoro.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane čovjeka. Ne preuzimamo odgovornost za nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
